### Notas antes de leer:
1. NO ESTA BIEN DOCUMENTADO. Justo cuando conseguí lo que necesitaba deje de documentar el proyecto, queda pendiente.
2. Logra combina bs4, selenium y pandas para sacar el df que quería en 11 segundos.
(De lo que habría tomado al menos 1 hora y media JAJAJAJ)
3. Aprendizaje de este project = Como usar el "select" method de selenium para los valores de
dropdown. (Como la selección de una fecha en este caso)
4. Tiempo tomado para lograrlo: 2horas con 54 minutos (2.9 horas). Menos 1.22 horas que me tomé con inversiones de Microwd. (Y eso que ni tocaba selenium en 40 días)
5. Este proyecto tiene información confidencial. Tener cuidado

## Paso 1: Sentar el driver.

- Nota: NO ACTUALIZES CHROME. El webdriver (El archivo necesario para correr selenium en chrome) solo funciona con una versión específica de chrome. Ten cuidado.

In [110]:
import html5lib
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait #This and the other one waits
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select # Para obtener todos los valores de todas las fechas
import time
from creds_ventas import USERNAME
from creds_ventas import PASSWORD

In [120]:
url_base = 'http://ventaenlinea.spsa.pe:8091/'

In [121]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(executable_path='./chromedriver',options=options)

## Iniciar sesión dentro de la página de ventas de intercorp.

In [122]:
# Entrar a la URL
driver.get(url_base)
# -Primera vez: Me pide usuario y contraseña.
# - Sí. Lo pide mientras no guardes contra ni usuario en el webdriver.

username = driver.find_element(By.XPATH,'//*[@id="txtLogin"]')
password = driver.find_element(By.XPATH,'//*[@id="txtPasw"]')

username.send_keys(USERNAME)
password.send_keys(PASSWORD)

driver.find_element_by_id("Button1").click()

## Entrar al formato Mass

### Entrar al formato y ver las estadísticas de todas las tiendas

In [125]:
# Por si lo usamos justo cuando se creó un nuevo día
drp_fecha = Select(driver.find_element(By.TAG_NAME, 'select'))
all_dates = [date.get_attribute('value') for date in drp_fecha.options]
drp_fecha.select_by_value(all_dates[2])

In [126]:
# Seleccionar SPSA
driver.find_element(By.XPATH,'//*[@id="form1"]/div[3]/table[2]/tbody/tr[2]/td[1]/a').click()

In [115]:
# Seleccionar MASS
driver.find_element(By.XPATH,'//*[@id="form1"]/div[3]/table[2]/tbody/tr[5]/td[1]/a').click()

### Obtener todas las estadísticas de todas las tiendas solo hoy día.

In [116]:
def parse_table(table_soup):
    data_for_df = []
    table_body = table_soup.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data_for_df.append([ele for ele in cols if ele]) # Get rid of empty values
    return data_for_df

In [ ]:
# Bien. Ahora nos va a desplegar un listado de todos los Mass que están registrados.
# El objetivo es obtener los metadatos de todos los Mass pero de TODOS los días posibles.
# Para realizar esto primero me voy a enfocar a obtener todos estos datos de almenos este día.

In [47]:
# Obtiene los datos de todas
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
table_soup = soup.find("table", {"id":"datos"})
data_all_sucursales = parse_table(table_soup)

In [95]:
headers = ('Sucursal', 'Vta Nta', 'Meta', 'Cump%', 'Trxs', 'Vta LY', 'LY%')

In [96]:
df = pd.DataFrame(data_all_sucursales, columns = headers)
# Mala (La primera tienda de la lista) aparece como el segundo elemento aquí.
# Loreto (La última tienda de la lista) aparece como el antepenultimo elemento aquí.
df = df[1:-3] # Así que recortamos nuestra lista

In [97]:
df['Day'] = 'Holis' # Perfecto. Funciona genial

### Ahora vemos como sacarlo para todos los días.

In [78]:
drp_fecha = Select(driver.find_element(By.TAG_NAME, 'select')) # Esto permite obtener los valores
# del dropdown de las fechas
drp_fecha

In [85]:
# Esto obtiene todas las fechas
drp_fecha = Select(driver.find_element(By.TAG_NAME, 'select'))
all_dates = [date.get_attribute('value') for date in drp_fecha.options]
len(all_dates)

31

In [ ]:
# Esto carga todos los datos de cada día en el listado
for date in all_dates:
    drp_fecha = Select(driver.find_element(By.TAG_NAME, 'select'))
    drp_fecha.select_by_value(date)
    print(date)

In [118]:
lister_dfs = []
headers = ('Sucursal', 'Vta Nta', 'Meta', 'Cump%', 'Trxs', 'Vta LY', 'LY%')
start = time.process_time()
# Esto carga todos los datos de cada día en el listado
for date in all_dates:
    drp_fecha = Select(driver.find_element(By.TAG_NAME, 'select'))
    drp_fecha.select_by_value(date)
    print("Obteniendo datos de:", date)
    ###
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    table_soup = soup.find("table", {"id":"datos"})
    data_all_sucursales = parse_table(table_soup)
    df = pd.DataFrame(data_all_sucursales, columns = headers)
    # Mala (La primera tienda de la lista) aparece como el segundo elemento aquí.
    # Loreto (La última tienda de la lista) aparece como el antepenultimo elemento aquí.
    df = df[1:-3] # Así que recortamos nuestra lista
    df['Day'] = date # Sentado día
    ###
    lister_dfs.append(df)
    print("Datos obtenidos con exito!")
print("LOGRADOOOO. Tiempo tomado:", time.process_time() - start)
final_df = pd.concat(lister_dfs,ignore_index=True)

In [ ]:
final_df